# Separate the bilby samples by SNR thresholds
- SNR network 8, indiv minimum 2
- SNR network 9, indiv minimum 2
- SNR network 10, indiv minimum 2

In [7]:
import pandas as pd
import os
import warnings

warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
cbcCatPath = "/global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/data/mockCBCCatalog_csvs/BBHs_0,aligned,sampledOnly.csv"  # Modify this
print("Loading CBC dictionary from", cbcCatPath)
cbcCat = pd.read_csv(cbcCatPath)
print("CBC DF Header:", cbcCat.columns.values)

import bilby as bb
import sys

sys.path.append("/global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/utils")
import utils as ut
import numpy as np

bb.core.utils.log.setup_logger(outdir=".", label=None, log_level=30)

Loading CBC dictionary from /global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/data/mockCBCCatalog_csvs/BBHs_0,aligned,sampledOnly.csv
CBC DF Header: ['galaxyID' 'redshiftHubble' 'ra' 'dec' 'mra' 'mdec' 'mass_ratio'
 'chirp_mass' 'luminosity_distance' 'theta_jn' 'psi' 'phase' 'chi_1'
 'chi_2']


In [8]:
cbcCat

,galaxyID,redshiftHubble,ra,dec,mra,mdec,mass_ratio,chirp_mass,luminosity_distance,theta_jn,psi,phase,chi_1,chi_2
0,10070200392465,2.787557,62.770779,-40.208431,172.823600,81.175717,0.843185,119.326237,23772.324284,2.299695,2.109767,2.628879,0.155300,0.060057
1,9814202166423,2.044943,62.394007,-37.236766,105.784619,41.289369,0.317438,116.323346,16252.753222,1.315168,2.866630,2.943843,0.229491,-0.768030
2,10072100184865,1.834209,67.617224,-41.572680,241.762863,70.242273,0.903382,63.699137,14204.132784,1.196491,0.925658,1.934347,0.116902,0.591217
3,9816101164643,1.442409,66.938860,-36.626004,330.745078,-75.967953,0.846207,113.757228,10528.900213,1.133533,1.809821,6.267067,0.835139,0.251175
4,10070201792402,2.190634,61.436562,-41.042127,92.144000,-15.350534,0.727091,93.887872,17693.717694,1.542628,2.591319,2.455860,0.171522,0.526389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9999995,9686201071622,2.492613,62.544190,-35.369961,185.429306,-39.294575,0.948780,59.376173,20736.583191,1.164657,1.187009,5.997527,0.346023,-0.272609
9999996,9816101815157,1.173380,67.224608,-37.116259,316.626212,-3.677308,0.755705,99.449215,8131.795058,2.472627,2.756257,1.968942,0.259125,-0.082369
9999997,10070100052626,1.902584,61.655855,-40.896745,173.349455,84.804486,0.496560,94.427943,14863.916642,1.793663,1.804905,3.113470,-0.085643,-0.002161
9999998,9814201794489,2.171678,60.824988,-37.371238,98.081873,83.720684,0.819102,77.788407,17505.164617,1.806159,2.404912,0.625993,-0.257656,0.067172


In [9]:
priorPath = "/pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/bilby/gw/prior_files/bbh_aligned_gwtc.prior"  # Modify this

print("Loading prior from", priorPath)
prior = bb.gw.prior.ConditionalPriorDict(priorPath)
cbcKeys = prior.sample().keys()
print("Prior keys:", cbcKeys)

Loading prior from /pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/bilby/gw/prior_files/bbh_aligned_gwtc.prior
Prior keys: dict_keys(['mass_ratio', 'chirp_mass', 'luminosity_distance', 'dec', 'ra', 'theta_jn', 'psi', 'phase', 'chi_1', 'chi_2'])


In [10]:
ifos = bb.gw.detector.InterferometerList(["H1", "V1", "L1"])
duration = 8  # Modify this
sampling_frequency = 2048
waveform_arguments = dict(
    waveform_approximant="IMRPhenomXP",  # waveform approximant name
    reference_frequency=50.0,  # gravitational waveform reference frequency (Hz)
)

In [5]:
fname = "batch_0_bbh_aligned.txt"
dataPath = "/global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/data"

In [6]:
try:
    del lastRow, l
except:
    pass

if not os.path.isfile(os.path.join(dataPath, fname)):
    open(os.path.join(dataPath, fname), "a").close()  #
else:
    with open(os.path.join(dataPath, fname), "r") as f:
        try:
            for l in f.readlines():
                pass
            lastRow = int(l.split(",")[0])
            print("Last row set as {}".format(lastRow))
        except:
            print("Unable to read last line, setting last row index to -1")
            lastRow = -1

Last row set as 2312361


In [ ]:
net_snr = []
indiv_snr = []
print("Computing SNR")
for ids, row in cbcCat.iloc[lastRow + 1 :].iterrows():
    injDict = {}
    for key in cbcKeys:
        if key in ("ra", "dec"):
            injDict[key] = row["m" + key]
        else:
            injDict[key] = row[key]
    injDict["geocent_time"] = ut.fakeGeoCentTime()

    waveform_generator = bb.gw.waveform_generator.WaveformGenerator(
        sampling_frequency=sampling_frequency,
        duration=duration,
        frequency_domain_source_model=ut.get_source_model("BBH"),
        parameters=injDict,
        waveform_arguments=waveform_arguments,
        start_time=injDict["geocent_time"] - duration / 2,
    )

    ifos.set_strain_data_from_power_spectral_densities(  # Set the strain
        duration=duration,
        sampling_frequency=sampling_frequency,
        start_time=injDict["geocent_time"] - duration / 2,
    )

    # Compute SNR

    network_snr = 0
    indivSNR = []
    for ifo in ifos:
        signal = ifo.get_detector_response(
            waveform_polarizations=waveform_generator.frequency_domain_strain(injDict),
            parameters=injDict,
        )
        single_snr_squared = ifo.optimal_snr_squared(signal).real
        indivSNR.append(np.sqrt(single_snr_squared))
        network_snr += single_snr_squared
    network_snr = np.sqrt(network_snr)

    # net_snr.append(network_snr)
    indiv_snr = np.min(indivSNR)

    with open(os.path.join(dataPath, fname), "a") as myfile:
        myfile.write(
            str([ids, network_snr, indiv_snr])[1:-1] + "\n"
        )  # index, netSnr, minIndivSNR

    if ids % (len(cbcCat) / 100) == 0:
        print("{}% complete".format(ids // (len(cbcCat) / 100)))

# cbcCat["Individual SNR minimum"] = indiv_snr
# cbcCat["Network SNR"] = net_snr

Computing SNR


In [ ]:
# Open the file, read the three columns

# Add them to cbcCat

# Then save the file

In [52]:
SNRCuts = pd.read_csv(
    "../data/combined_batches.txt",
    names=["index", "networkSNR", "IndividualSNR1", "IndividualSNR2", "IndividualSNR3"],
)
SNRCuts["IndividualSNR1"] = [float(x.split("[")[-1]) for x in SNRCuts["IndividualSNR1"]]
SNRCuts["IndividualSNR3"] = [float(x.split("]")[0]) for x in SNRCuts["IndividualSNR3"]]
SNRCuts["minSNR"] = [
    np.min([x, y, z])
    for x, y, z in zip(
        SNRCuts["IndividualSNR1"], SNRCuts["IndividualSNR2"], SNRCuts["IndividualSNR3"]
    )
]
SNRCuts.drop_duplicates("index", ignore_index=True, inplace=True)
SNRCuts.sort_values("index", inplace=True)

In [53]:
SNRCuts

,index,networkSNR,IndividualSNR1,IndividualSNR2,IndividualSNR3,minSNR
0,0,2.473854,1.182990,1.561964,1.510217,1.182990
1,1,1.844235,1.087657,0.465239,1.414835,0.465239
2,2,2.398134,0.992021,2.105251,0.578668,0.578668
3,3,6.349467,4.849165,0.988400,3.977989,0.988400
4,4,0.985572,0.230451,0.937080,0.200314,0.200314
...,...,...,...,...,...,...
9999995,9999995,1.961467,1.313095,1.065746,0.993641,0.993641
9999996,9999996,7.672122,4.671259,2.357164,5.611111,2.357164
9999997,9999997,1.842920,0.942448,0.645582,1.446157,0.645582
9999998,9999998,1.769257,0.646848,1.616645,0.313558,0.313558


In [56]:
cbcCat["Network SNR"] = SNRCuts["networkSNR"]
cbcCat["minimum individual SNR"] = SNRCuts["minSNR"]

In [58]:
outPath = "/global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/data/mockCBCCatalog_csvs/BBHs_0,aligned,sampledPlusSNR.csv"  # Modify this
cbcCat.to_csv(outPath, index=False)
print("Finished, csv written to", outPath)

Finished, csv written to /global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/data/mockCBCCatalog_csvs/BBHs_0,aligned,sampledPlusSNR.csv
